<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=844483b7a7cd7cd122eedd83876b76e38ac543160ceec2dc2012d09903b14b34
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 13:17:40
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.38 C
-------------------
Today PnL: -2.10 L (-1.5%)
Current PnL: -23.33 L (-15.54%)
CY Booked + Current PnL: -11.43 L (-7.62%)
-------------------
Total profit:  1.06 L
Total loss:  -24.40 L
-------------------
Total Booked + Current PnL: 15.55 L (12.63%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.61%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 90.54 L (65.53%)
Deployed:  1.23 C
Current:  1.38 C
CAGR/XIRR %: 6.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,98.29,53.0,M-SC,1.26,85425.0,-15352.0,15445.0,2.31,-15.23,18.08,0.09,245.0,-0.99,0.62,12.23,OX40N,NTT,DURABLES
18,CIPLA,1662.74,-20.81,52.0,X-LC,3.72,211281.0,6781.0,16522.0,-1.37,3.32,7.82,11.39,10.0,0.41,1.54,12.56,X40N,ATH,PHARMA
83,VOLTAS,1530.00,-1.08,52.0,X-MC,1.01,207600.0,15858.0,21902.0,0.40,8.27,10.55,19.69,99.0,0.72,1.51,15.63,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,46.0,X-LC,2.85,214172.0,3806.0,24972.0,-0.15,1.81,11.66,13.68,37.0,0.15,1.56,18.63,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,39.0,X-LC,2.75,194378.0,-5760.0,26649.0,-0.44,-2.88,13.71,10.44,4.0,-0.22,1.42,2.53,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,98.29,53.0,M-SC,1.26,85425.0,-15352.0,15445.0,2.31,-15.23,18.08,0.09,245.0,-0.99,0.62,12.23,OX40N,NTT,DURABLES
66,SONACOMS,1006.0,-37.48,60.0,M-SC,12.94,77822.0,-23337.0,98227.0,2.05,-23.07,126.22,74.03,202.0,-0.24,0.57,9.74,AR,BTT,AUTO
34,ICICIPRULI,790.0,-20.88,51.0,X-MC,1.93,136176.0,388.0,42364.0,1.51,0.29,31.11,31.48,107.0,0.01,0.99,12.44,X40,ATH,INSURANCE
85,WIPRO,420.0,-12.43,53.0,M-LC,5.95,153636.0,2691.0,106762.0,1.09,1.78,69.49,72.51,53.0,0.03,1.12,7.91,XR,NTT,IT
4,ANGELONE,3033.0,9.18,61.0,X-SC,10.37,193520.0,2514.0,55192.0,0.82,1.32,28.52,30.21,157.0,0.05,1.41,22.47,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-53.17,12.0,X-LC,1.01,147941.0,-128582.0,238658.0,-38.32,-46.50,161.32,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
81,VALIANTORG,838.0,-277.79,23.0,H-SC,3.11,98702.0,-64903.0,186221.0,-4.98,-39.67,188.67,74.15,139.0,-0.35,0.72,26.52,XR,NTT,CHEMICALS
10,BANDHANBNK,400.0,-13.31,47.0,H-SC,2.95,217093.0,-61466.0,318910.0,-3.84,-22.07,146.90,92.42,151.0,-0.19,1.58,25.46,XY24,NTT,BANKS
23,DEN,75.0,89.12,37.0,M-SC,3.86,53056.0,-31408.0,66946.0,-3.66,-37.19,126.18,42.07,237.0,-0.47,0.39,12.67,AR,NTT,ENTERTAINMENT
68,SURYODAY,240.0,57.89,54.0,H-SC,10.48,145860.0,-33211.0,98937.0,-2.97,-18.55,67.83,36.71,135.0,-0.34,1.06,44.72,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.3,124073.0,737.0,130885.0,-2.40,0.60,105.49,106.72,119.0,0.01,0.91,24.12,AR,ATH,MISC
54,RECLTD,446.00,45.36,43.0,M-LC,6.2,203720.0,550.0,41579.0,-0.19,0.27,20.41,20.74,55.0,0.01,1.49,5.83,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,104.20,57.0,H-SC,11.55,129141.0,-13338.0,32853.0,-1.87,-9.36,25.44,13.70,163.0,-0.41,0.94,52.42,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.29,53.0,M-SC,1.26,85425.0,-15352.0,15445.0,2.31,-15.23,18.08,0.09,245.0,-0.99,0.62,12.23,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220626.0,-49041.0,85382.0,-0.23,-18.19,38.70,13.47,138.0,-0.57,1.61,12.18,XY24,NTT,PAINTS
17,CERA,9475.0,-23.87,39.0,H-SC,1.99,138897.0,-37006.0,79032.0,0.45,-21.04,56.90,23.89,149.0,-0.47,1.01,19.96,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.72,43.0,H-MC,7.00,101868.0,-29967.0,72194.0,0.18,-22.73,70.87,32.03,98.0,-0.42,0.74,14.10,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203720.0,550.0,41579.0,-0.19,0.27,20.41,20.74,55.0,0.01,1.49,5.83,XY25,NTT,FINANCE
31,HINDZINC,730.22,34.91,67.0,M-LC,8.57,220328.0,15252.0,97319.0,0.19,7.44,44.17,54.89,52.0,0.16,1.61,33.32,X5K,ATH,METALS
85,WIPRO,420.00,-12.43,53.0,M-LC,5.95,153636.0,2691.0,106762.0,1.09,1.78,69.49,72.51,53.0,0.03,1.12,7.91,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,51.0,M-LC,9.50,155814.0,14974.0,29511.0,-2.04,10.63,18.94,31.59,66.0,0.51,1.14,39.94,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,57.0,H-MC,12.65,190881.0,11073.0,101339.0,-2.17,6.16,53.09,62.52,88.0,0.11,1.39,42.34,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,67.0,M-LC,8.57,220328.0,15252.0,97319.0,0.19,7.44,44.17,54.89,52.0,0.16,1.61,33.32,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,57.0,H-MC,12.65,190881.0,11073.0,101339.0,-2.17,6.16,53.09,62.52,88.0,0.11,1.39,42.34,XR,NTT,BANKS
85,WIPRO,420.00,-12.43,53.0,M-LC,5.95,153636.0,2691.0,106762.0,1.09,1.78,69.49,72.51,53.0,0.03,1.12,7.91,XR,NTT,IT
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.30,124073.0,737.0,130885.0,-2.40,0.60,105.49,106.72,119.0,0.01,0.91,24.12,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-48.62,30.0,M-SC,3.40,86790.0,-4708.0,83457.0,-0.60,-5.15,96.16,86.07,223.0,-0.06,0.63,26.03,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.17,12.0,X-LC,1.01,147941.0,-128582.0,238658.0,-38.32,-46.50,161.32,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-9.01,28.0,X-MC,1.38,225529.0,-22202.0,97271.0,-1.86,-8.96,43.13,30.30,92.0,-0.23,1.65,3.14,X40,ATH,ELECTRICAL
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175808.0,-24698.0,121853.0,-1.40,-12.32,69.31,48.45,90.0,-0.20,1.28,1.38,X40N,ATH,FINANCE
51,QUESS,986.00,-48.21,30.0,X-SC,37.84,51498.0,-13508.0,164438.0,-1.40,-20.78,319.31,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
5,ASIANPAINT,3465.66,-19.07,31.0,X-LC,5.31,204160.0,-47608.0,100814.0,-0.96,-18.91,49.38,21.13,27.0,-0.47,1.49,9.82,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-4.11,40.0,H-LC,0.90,154165.0,-31930.0,79410.0,0.19,-17.16,51.51,25.51,15.0,-0.40,1.12,13.61,AR,ATH,ELECTRICAL
72,TATAMOTORS,1065.00,-53.17,12.0,X-LC,1.01,147941.0,-128582.0,238658.0,-38.32,-46.50,161.32,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
83,VOLTAS,1530.00,-1.08,52.0,X-MC,1.01,207600.0,15858.0,21902.0,0.40,8.27,10.55,19.69,99.0,0.72,1.51,15.63,XY25,NTT,AC
28,HAVELLS,2069.16,-9.01,28.0,X-MC,1.38,225529.0,-22202.0,97271.0,-1.86,-8.96,43.13,30.30,92.0,-0.23,1.65,3.14,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.58,42.0,X-MC,1.47,136752.0,619.0,30509.0,0.80,0.45,22.31,22.86,91.0,0.02,1.00,13.13,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,37.84,51498.0,-13508.0,164438.0,-1.40,-20.78,319.31,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-53.17,12.0,X-LC,1.01,147941.0,-128582.0,238658.0,-38.32,-46.50,161.32,39.81,54.0,-0.54,1.08,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175808.0,-24698.0,121853.0,-1.40,-12.32,69.31,48.45,90.0,-0.20,1.28,1.38,X40N,ATH,FINANCE
82,VBL,671.64,-21.55,39.0,X-LC,10.49,282698.0,-33144.0,145816.0,0.15,-10.49,51.58,35.67,5.0,-0.23,2.06,1.91,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.86,35.0,X-MC,7.69,219900.0,-43465.0,156811.0,-1.00,-16.50,71.31,43.04,84.0,-0.28,1.60,2.09,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,37.84,51498.0,-13508.0,164438.0,-1.40,-20.78,319.31,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-41.04,34.0,X-SC,3.33,80314.0,-64846.0,140774.0,-1.10,-44.67,175.28,52.31,136.0,-0.46,0.59,7.50,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.44,34.0,X-SC,4.35,93541.0,-35129.0,80427.0,-1.91,-27.30,85.98,35.20,219.0,-0.44,0.68,7.36,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.71,44.0,X-SC,1.05,106134.0,4130.0,39800.0,-2.01,4.05,37.50,43.06,122.0,0.10,0.77,21.86,X40N,ATH,MISC
32,HONAUT,58357.33,-23.37,43.0,X-SC,2.23,108045.0,-19893.0,67031.0,-0.39,-15.55,62.04,36.84,143.0,-0.30,0.79,10.61,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-30.29,43.0,X-LC,12.30,278212.0,-67744.0,135990.0,-1.58,-19.58,48.88,19.73,1.0,-0.50,2.03,2.47,X40,ATH,IT
39,INFY,2275.00,-18.52,51.0,X-LC,7.50,317349.0,4461.0,167211.0,-0.22,1.43,52.69,54.87,3.0,0.03,2.32,8.17,X40,BTT,IT
41,ITC,452.00,-39.61,39.0,X-LC,2.75,194378.0,-5760.0,26649.0,-0.44,-2.88,13.71,10.44,4.0,-0.22,1.42,2.53,X40,NTT,FMCG
82,VBL,671.64,-21.55,39.0,X-LC,10.49,282698.0,-33144.0,145816.0,0.15,-10.49,51.58,35.67,5.0,-0.23,2.06,1.91,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,40.0,H-LC,7.29,245496.0,-16123.0,135342.0,-0.64,-6.16,55.13,45.57,7.0,-0.12,1.79,4.46,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.72,80986.0,-32563.0,72612.0,-0.96,-28.68,89.66,35.27,268.0,-0.45,0.59,98.21,XR,NTT,HOTELS
48,MASFIN,398.61,-21.86,37.0,H-SC,9.90,88905.0,-9075.0,30681.0,-1.82,-9.26,34.51,22.05,152.0,-0.30,0.65,29.23,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,104.20,57.0,H-SC,11.55,129141.0,-13338.0,32853.0,-1.87,-9.36,25.44,13.70,163.0,-0.41,0.94,52.42,OX40N,NTT,BANKS
60,SAMMAANCAP,326.00,-184.09,65.0,M-SC,17.40,145800.0,-3420.0,147593.0,-0.25,-2.29,101.23,96.62,208.0,-0.02,1.06,57.79,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.48,145860.0,-33211.0,98937.0,-2.97,-18.55,67.83,36.71,135.0,-0.34,1.06,44.72,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-184.09,65.0,M-SC,17.40,145800.0,-3420.0,147593.0,-0.25,-2.29,101.23,96.62,208.0,-0.02,1.06,57.79,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.48,145860.0,-33211.0,98937.0,-2.97,-18.55,67.83,36.71,135.0,-0.34,1.06,44.72,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,67.0,M-LC,8.57,220328.0,15252.0,97319.0,0.19,7.44,44.17,54.89,52.0,0.16,1.61,33.32,X5K,ATH,METALS
50,PGHH,17907.65,-26.58,60.0,X-MC,2.90,212655.0,11835.0,55950.0,-0.01,5.89,26.31,33.76,80.0,0.21,1.55,11.40,X40,ATH,FMCG
67,STARHEALTH,761.00,15.38,61.0,H-SC,8.10,256850.0,-24273.0,157141.0,-1.57,-8.63,61.18,47.26,144.0,-0.15,1.87,37.61,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.49
1,25,44.01
2,50,75.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.37
LC    33.72
MC    21.84
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.70
X40      14.85
XY25     12.09
XR       10.96
X40N     10.08
AR        8.22
OX40N     7.68
X200      1.80
X5K       1.61
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.62
X-LC    22.62
X-MC    16.10
M-SC    12.69
M-LC     5.36
H-LC     4.71
X-SC     4.62
H-MC     3.80
M-MC     1.62
L-SC     1.44
L-LC     1.03
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,-6.54,41.65
IT,13.06,-17.42,81.03
FINANCE,12.15,-13.45,64.88
MISC,6.92,-21.05,88.12
BANKS,6.43,-13.13,72.65
PAINTS,5.61,-19.91,37.78
ELECTRICAL,5.35,-12.29,51.70
INSURANCE,3.86,-4.39,43.42
AUTO,2.85,-47.30,117.72


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3363629.0,22
XR,1405919.0,14
AR,1203156.0,9
X40,805328.0,10
X40N,644712.0,8
OX40N,590171.0,10
XY25,525199.0,8
SR,268351.0,2
X5K,97319.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3485000.0,25
M-SC,1299320.0,15
X-LC,1141697.0,13
X-MC,1117037.0,11
X-SC,547662.0,6
H-MC,358201.0,3
H-LC,292841.0,3
M-LC,275171.0,4
L-SC,263168.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1210958.0      6
           XR         832293.0      7
           AR         823231.0      5
M-SC       XY24       703888.0      6
X-MC       XY24       590377.0      4
X-LC       X40        498807.0      5
           XY24       308631.0      2
X-SC       X40N       302797.0      4
H-SC       OX40N      278384.0      4
           SR         268351.0      2
X-MC       X40        226094.0      4
X-LC       X40N       220062.0      3
H-LC       AR         214752.0      2
H-MC       XY24       184668.0      1
X-MC       XY25       178713.0      2
L-SC       XR         173014.0      2
M-MC       XY24       171158.0      1
M-SC       AR         165173.0      2
X-SC       XY24       164438.0      1
M-SC       OX40N      149439.0      4
           XY25       147593.0      1
           XR         133227.0      2
X-MC       X40N       121853.0      1
X-LC       XY25       114197.0      3
M-LC       XR         106762.0      1
H-MC       XR         101339.0      1
M-LC       X5K         97319.0      1
L-SC       OX40N       90154.0      1
X-SC       X40         80427.0      1
H-LC       X200        78089.0      1
H-MC       OX40N       72194.0      1
H-SC       MH          71783.0      1
L-MC       XR          59284.0      1
L-LC       XY25        43117.0      1
M-LC       XY25        41579.0      1
           XY24        29511.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
